# Análise Completa - Case Ifood: Teste A/B Estratégia de Cupons

Notebook único para orquestrar as tarefas de execução de *setup*, **ETL** e análise dos dados, integrando os diferentes módulos do repositório de origem:

- Clona/atualiza o repositório do projeto, com as dependências, no Colab
- Instala dependências e faz o **download** dos dados brutos
- Sobe Spark e executa o **ETL** (orders/consumers/restaurants + mapa A/B)
- Mantém `orders_silver` e `users_silver` em memória
- Realiza a análise exploratória dos dados


## Configuração do Ambiente e Preparação dos Dados

### Configuração de Ambiente e Download de Dados Brutos

In [ ]:
import os, sys, subprocess
from pathlib import Path

GITHUB_USER = "silvaniacorreia"
REPO_NAME   = "ifood-case-cupons"
REPO_URL    = f"https://github.com/{GITHUB_USER}/{REPO_NAME}.git"

def run(cmd):
    print(">", " ".join(cmd))
    subprocess.check_call(cmd)

# 1) clonar/atualizar repositório
ROOT = Path("/content")
PROJECT_DIR = ROOT / REPO_NAME
if not PROJECT_DIR.exists():
    run(["git", "clone", REPO_URL, str(PROJECT_DIR)])
else:
    os.chdir(PROJECT_DIR)
    run(["git", "fetch", "--all"])
    run(["git", "reset", "--hard", "origin/main"])
    run(["git", "checkout", "main"])
    run(["git", "pull", "origin", "main"])
os.chdir(PROJECT_DIR)

# 2) deps + download programático
run([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "--no-cache-dir"])
run([sys.executable, "scripts/download_data.py"])

# 3) sys.path
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))
print("✔️ Bootstrap concluído. Projeto:", PROJECT_DIR)

### Iniciando o Spark

In [ ]:
from src.utils import load_settings, get_spark

s = load_settings()  
extra = dict(getattr(s.runtime.spark, "conf", {}) or {})
extra.setdefault("spark.sql.execution.arrow.pyspark.enabled", "true")

spark = get_spark(
    app_name=s.runtime.spark.app_name,
    shuffle_partitions=s.runtime.spark.shuffle_partitions,
    extra_conf=extra,
)
print("✔️ Spark ativo - versão:", spark.version)
spark.range(5).show()

### Análises Pré-Flight

In [ ]:
## Checagens dados brutos
from src.checks import preflight
from pprint import pprint

rep = preflight(s.data.raw_dir, strict=False)
print("Pré-flight (resumo):")
pprint({
    "raw_dir": rep["raw_dir"],
    "orders_format_guess": rep["orders_format_guess"],
    "files": {k: {kk: vv for kk, vv in v.items() if kk in ("exists","size_bytes","gzip_ok","tar_ok")} for k, v in rep["files"].items()},
    "ab_csv_candidates": rep["ab_csv_candidates"][:3],
})


### ETL (Extração, Transformação e Carga)

In [ ]:
from src import etl, checks
from pyspark.sql import functions as F
import os

def _get_exp_window(s):
    """
    Lê a janela do experimento a partir das configurações. Caso não exista, utiliza inferência automática.

    Parâmetros:
        s: Objeto de configurações carregado.

    Retorna:
        Tuple[str, str, bool]: Data de início, data de fim e flag de inferência automática.
    """
    win = getattr(s.analysis, "experiment_window", None)
    if isinstance(win, dict):
        start = win.get("start")
        end   = win.get("end")
    else:
        start = None
        end   = None
    auto = bool(getattr(s.analysis, "auto_infer_window", True))
    return start, end, auto

start, end, auto = _get_exp_window(s)

# 1) Leitura dos dados brutos
orders, consumers, restaurants, abmap = etl.load_raw(spark, s.data.raw_dir)
checks.profile_loaded(orders, consumers, restaurants, abmap, n=5)

# 2) Limpeza e conformidade dos dados
# Inclui normalização de timezone e aplicação de janela experimental
# Utiliza quantis para robustez contra outliers
df = etl.clean_and_conform(
    orders, consumers, restaurants, abmap,
    business_tz=getattr(s.analysis, "business_tz", "America/Sao_Paulo"),
    treat_is_target_null_as_control=getattr(s.analysis, "treat_is_target_null_as_control", False),
    experiment_start=start,
    experiment_end=end,
    auto_infer_window=auto,
    use_quantile_window=True,     
    verbose=True
)

# 3) Ajustes finais e agregações para análise
orders_silver = etl.build_orders_silver(df)
orders_silver = etl.enrich_orders_for_analysis(orders_silver)
users_silver  = etl.build_user_aggregates(orders_silver, start, end)

# 4) Cálculo de recência com base no último timestamp observado
ref_ts = orders_silver.agg(F.max("event_ts_utc")).first()[0]
users_silver = users_silver.withColumn("recency", F.datediff(F.lit(ref_ts), F.col("last_order")))

# 5) Salvar resultados em formato Parquet (opcional)
SAVE_PARQUET = False
if SAVE_PARQUET:
    (
        orders_silver
        .write
        .mode("overwrite")
        .partitionBy("event_date_brt")
        .parquet(f"{s.data.processed_dir}/orders_silver.parquet")
    )
    users_silver.write.mode("overwrite").parquet(f"{s.data.processed_dir}/users_silver.parquet")

# 6) Contagem de linhas para validação
print("orders_silver:", orders_silver.count(), "linhas")
print("users_silver :", users_silver.count(), "linhas")

# 7) Exibição de amostras para validação
try:
    from IPython.display import display
    display(orders_silver.limit(5).toPandas())
    display(users_silver.limit(5).toPandas())
except Exception as e:
    print("Aviso: toPandas falhou, mostrando via Spark .show()")
    orders_silver.show(5, truncate=False)
    users_silver.show(5, truncate=False)

### Checagem dos Dados

Foram investigadas duplicatas semânticas na fato (IDs diferentes com mesmo cliente/loja/tempo/valor). Como apenas 1 caso foi encontrado, o que gera efeito desprezível, não foi aplicada a deduplicação adicional.

In [ ]:
from pyspark.sql import functions as F

def check_post_etl(
    orders_silver,
    users_silver,
    *,
    light: bool = True,
    key_cols: list[str] | None = None,
    sample_frac: float = 0.001,
    preview_rows: int = 5,
    use_pandas_preview: bool = False,
    check_semantic_dups: bool = True,
):
    """
    Executa checagens leves pós-ETL para registro no Colab.
    - light=True: nulos apenas em colunas-chave e previews por sample.
    - light=False: nulos em todas as colunas (lento).
    - check_semantic_dups: investiga duplicatas semânticas na fato (lento moderado).
    """
    if key_cols is None:
        key_cols = [
            "order_id", "customer_id", "merchant_id",
            "event_ts_utc", "order_total_amount",
            "is_target", "price_range", "language", "active",
            "delivery_time_imputed", "minimum_order_value_imputed",
        ]
    key_cols = [c for c in key_cols if c in orders_silver.columns]

    print("Faixa de datas (UTC) em orders_silver:")
    orders_silver.agg(
        F.min("event_ts_utc").alias("min_utc"),
        F.max("event_ts_utc").alias("max_utc"),
    ).show(truncate=False)

    print("Split A/B (users):")
    users_silver.groupBy("is_target").count().orderBy("is_target").show()

    # --- Nulos em orders_silver ---
    def nulls_by_col(df, cols):
        exprs = [F.sum(F.col(c).isNull().cast("int")).alias(c) for c in cols]
        return df.select(exprs)

    if light:
        print(f"Nulos (colunas-chave): {key_cols}")
        nulls_by_col(orders_silver, key_cols).show(truncate=False)
    else:
        print("Nulos (todas as colunas) — operação pesada:")
        nulls_by_col(orders_silver, orders_silver.columns).show(truncate=False)

    # --- Duplicatas semânticas (order_ids diferentes com mesmo cliente/restaurante/ts/valor) ---
    if check_semantic_dups:
        print("\nPossíveis duplicatas sistêmicas (mesmo cliente/restaurante/ts/valor, order_id distinto):")
        dups = (
            orders_silver
            .groupBy("customer_id", "merchant_id", "event_ts_utc", "order_total_amount")
            .agg(
                F.countDistinct("order_id").alias("n_orders"),
                F.collect_set("order_id").alias("order_ids"),
            )
            .filter(F.col("n_orders") > 1)
        )
        total_dups = dups.count()
        print(f"Total de combinações com múltiplos order_id: {total_dups}")
        if total_dups > 0:
            dups.select("customer_id","merchant_id","event_ts_utc","order_total_amount","n_orders","order_ids")\
                .orderBy(F.col("n_orders").desc())\
                .show(10, truncate=False)

    # --- Previews rápidos ---
    print("\nPreview orders_silver (sample leve):")
    orders_preview_cols = [c for c in [
        "price_range","order_id","customer_id","merchant_id",
        "event_ts_utc","order_total_amount","origin_platform",
        "is_target","language","active"
    ] if c in orders_silver.columns]
    preview_df = orders_silver.sample(False, sample_frac, seed=42).select(*orders_preview_cols)
    if preview_df.rdd.isEmpty():
        preview_df = orders_silver.select(*orders_preview_cols).limit(preview_rows)
    preview_df.show(preview_rows, truncate=False)

    print("\nPreview users_silver (primeiras linhas):")
    users_preview_cols = [c for c in [
        "customer_id","last_order","frequency","monetary","is_target","recency"
    ] if c in users_silver.columns]
    users_silver.select(*users_preview_cols).show(preview_rows, truncate=False)

    if use_pandas_preview:
        try:
            from IPython.display import display
            display(orders_silver.limit(preview_rows).toPandas())
            display(users_silver.limit(preview_rows).toPandas())
        except Exception:
            pass

# --- Executar em modo leve ---
check_post_etl(orders_silver, users_silver, light=True, check_semantic_dups=True)


## A/B de cupons

### Importações e Configurações

In [ ]:
from pyspark.sql import functions as F
from src.analysis_ab import (
    compute_ab_summary,
    compute_robust_metrics,
    collect_user_level_for_tests,
    run_ab_tests,
    run_nonparam_tests,
    financial_viability
)
from src.utils import load_settings
from src.analysis_ab import compute_ab_summary, collect_user_level_for_tests, compute_robust_metrics
from src.viz_ab import plot_group_bars, plot_ab_box, plot_ab_hist_overlay, save_table_csv

settings = load_settings("config/settings.yaml")



### Visualizações para exploração

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---------------------------
# 1) Amostragem controlada já no Spark (mais leve no driver)
#    - diminua sample_frac se ainda ficar pesado
#    - max_rows garante teto absoluto no tamanho coletado
# ---------------------------
SAMPLE_FRAC = 0.15     # 10–20% costuma bastar para ver a assimetria
MAX_ROWS    = 100_000  # teto duro (evita coletar demais)
CLIP_P      = 0.01     # recorta 1% de cada cauda para visual

users_pdf = collect_user_level_for_tests(
    users_silver.select("user_id","is_target","monetary","frequency"),  
    sample_frac=SAMPLE_FRAC,
    seed=42
)

# teto de linhas no pandas (caso a fração ainda gere um DF muito grande)
if len(users_pdf) > MAX_ROWS:
    users_pdf = users_pdf.sample(n=MAX_ROWS, random_state=42)

# AOV (se já não vier calculado)
if "aov_user" not in users_pdf.columns:
    users_pdf["aov_user"] = users_pdf["monetary"] / users_pdf["frequency"].replace({0: np.nan})

# ---------------------------
# 2) Recorte de cauda para visual (não altera dados de análise “oficial”)
# ---------------------------
q_low  = users_pdf[["monetary","frequency","aov_user"]].quantile(CLIP_P)
q_high = users_pdf[["monetary","frequency","aov_user"]].quantile(1-CLIP_P)
for col in ["monetary","frequency","aov_user"]:
    users_pdf[col] = users_pdf[col].clip(q_low[col], q_high[col])

# labels legíveis
users_pdf["grupo"] = users_pdf["is_target"].map({0:"Controle", 1:"Tratamento"}).astype("category")

# ---------------------------
# 3) Boxplots (com showfliers=False para não pesar na renderização)
# ---------------------------
sns.set_theme()
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.boxplot(x="grupo", y="monetary", data=users_pdf, ax=axes[0], showfliers=False)
axes[0].set_title("GMV por Usuário (amostra, caudas recortadas)")

sns.boxplot(x="grupo", y="frequency", data=users_pdf, ax=axes[1], showfliers=False)
axes[1].set_title("Pedidos por Usuário (amostra)")

sns.boxplot(x="grupo", y="aov_user", data=users_pdf, ax=axes[2], showfliers=False)
axes[2].set_title("AOV por Usuário (amostra)")
plt.tight_layout()
plt.show()

# ---------------------------
# 4) Barras de médias (na amostra) — registro do "primeiro olhar"
# ---------------------------
metrics_means = (users_pdf
                 .groupby("grupo")[["monetary","frequency","aov_user"]]
                 .mean()
                 .reset_index())
ax = metrics_means.plot(x="grupo", kind="bar", figsize=(10,6))
plt.title("Médias por Grupo (GMV, Pedidos, AOV) — amostra")
plt.ylabel("Valor médio (amostra)")
plt.tight_layout()
plt.show()

# ---------------------------
# 5) Histogramas (sobrepostos) com caudas recortadas
# ---------------------------
plt.figure(figsize=(10,6))
sns.histplot(users_pdf.loc[users_pdf["grupo"]=="Controle","frequency"],
             bins=30, stat="density", alpha=0.5, label="Controle")
sns.histplot(users_pdf.loc[users_pdf["grupo"]=="Tratamento","frequency"],
             bins=30, stat="density", alpha=0.5, label="Tratamento")
plt.legend()
plt.title("Distribuição de Pedidos por Usuário — amostra (caudas recortadas)")
plt.tight_layout()
plt.show()

# (opcional) cache da amostra para reusar na sessão
# users_pdf.to_parquet("outputs/cache/users_pdf_sample.parquet", index=False)


### Métricas por grupo

Premissas:
* Valor do cupom: R$ 10,00
    *  Pago integralmente pelo iFood
* Taxa de conversão: 25%
* Take rate: 23%

Dada a distribuição assimétrica dos dados, com muitos outliers, evidenciada pelos gráficos, optou-se por utilizar métricas robustas (medianas, p95, heavy users) para a análise de impacto. Métricas robustas ajudam a evitar decisões enviesadas por outliers, garantindo que o ROI/LTV:CAC seja interpretado à luz do comportamento da maioria dos usuários. Métricas baseadas em médias também são apresentadas para comparação, mas com cautela, pois podem ser influenciadas por valores extremos. O relatório final incluirá somente métricas robustas e testes estatísticos apropriados.

In [ ]:
out_ab = "outputs/ab"

# Resumo por grupo (descrição em Spark)
ab_summary_spark = compute_ab_summary(users_silver)
ab_summary_spark.show(truncate=False)
ab_summary_pdf = ab_summary_spark.toPandas()
save_table_csv(ab_summary_pdf, out_ab, "ab_means")

# Métricas robustas (mediana, p95, heavy)
robust_spark = compute_robust_metrics_spark(users_silver, heavy_threshold=3)
robust_pdf   = robust_spark.toPandas()
display(robust_pdf)



### Testes de significância

In [ ]:
# Teste paramétrico
ttest_out = run_ab_tests(users_pdf) 

# Teste não-paramétrico
mw_out    = run_nonparam_tests(users_pdf)  

print("Welch t-test:", ttest_out)
print("Mann–Whitney:", mw_out)

### Viabilidade financeira

In [ ]:
from src.utils import load_settings
s = load_settings()

finance = financial_viability(
    users_pdf,
    take_rate=s.finance.take_rate,                
    coupon_cost=s.finance.coupon_cost_default,    
    redemption_rate=0.30
)
finance


### Visualizações para relatório

In [ ]:
from src.viz_ab import plot_group_bars, plot_ab_box, plot_ab_hist_overlay

# barras com médias
plot_group_bars(
    ab_summary_pdf.rename(columns={"aov":"aov_user"}),
    metrics=["gmv_user","pedidos_user","aov_user"],
    labels_map={"gmv_user":"GMV/usuário","pedidos_user":"Pedidos/usuário","aov_user":"AOV"},
    outdir="outputs/ab", fname="bars_means", title="Médias por grupo"
)

# barras com medianas (robust_pdf vindo do Spark)
plot_group_bars(
    robust_pdf.rename(columns={
        "median_gmv_user":"GMV mediano",
        "median_pedidos_user":"Pedidos medianos",
        "median_aov_user":"AOV mediano"
    })[["is_target","GMV mediano","Pedidos medianos","AOV mediano"]],
    metrics=["GMV mediano","Pedidos medianos","AOV mediano"],
    outdir="outputs/ab", fname="bars_medians", title="Métricas robustas por grupo"
)

# distribuições com users_pdf (amostra)
for metric in ["monetary","frequency","aov_user"]:
    plot_ab_box(users_pdf, metric, outdir="outputs/ab", fname=f"box_{metric}", clip_p=0.01)
    plot_ab_hist_overlay(users_pdf, metric, outdir="outputs/ab", fname=f"hist_{metric}", clip_p=0.01)


## Análise de Segmentos

### Construção de segmentos

In [ ]:
from src.analysis_segments import build_rfm_buckets, ab_metrics_by_segment
from src.viz_segments import to_pandas_spark, save_table_csv
from pyspark.sql import functions as F

# RFM
users_with_rfm = build_rfm_buckets(users_silver)

sample_ids_spark = spark.createDataFrame(
    users_pdf[["customer_id"]].drop_duplicates()
)
rfm_small = (
    users_with_rfm
    .join(sample_ids_spark, "customer_id", "inner")
    .select("customer_id","rfm_segment")
    .toPandas()
)
users_pdf = users_pdf.merge(rfm_small, on="customer_id", how="left")

# Heavy / New / Platform / RFM
ab_heavy = ab_metrics_by_segment(users_silver, segment_col="heavy_user")
ab_new   = ab_metrics_by_segment(users_silver, segment_col="is_new_customer")
ab_plat  = ab_metrics_by_segment(users_silver, segment_col="origin_platform")
ab_rfm   = ab_metrics_by_segment(users_with_rfm, segment_col="rfm_segment", top_k_segments=10)

ab_heavy_pd = to_pandas_spark(ab_heavy)
ab_new_pd   = to_pandas_spark(ab_new)
ab_plat_pd  = to_pandas_spark(ab_plat)
ab_rfm_pd   = to_pandas_spark(ab_rfm)

outdir = "outputs/segments"
save_table_csv(ab_heavy_pd, outdir, "ab_heavy_summary")
save_table_csv(ab_new_pd,   outdir, "ab_new_summary")
save_table_csv(ab_plat_pd,  outdir, "ab_platform_summary")
save_table_csv(ab_rfm_pd,   outdir, "ab_rfm_summary")
ab_heavy_pd, ab_new_pd.head(), ab_plat_pd.head(), ab_rfm_pd.head()


### Visualização de Segmentos

In [ ]:
from src.viz_segments import plot_bars_by_segment, plot_box_by_segment, plot_hist_by_segment

figdir = "outputs/figs_segments"

# 1) Barras de médias por segmento (GMV/user, pedidos/user, AOV)
metrics_cols = ["gmv_user", "pedidos_user", "aov"]
plot_bars_by_segment(ab_heavy_pd, "heavy_user", metrics_cols, title="Médias por segmento", outdir=figdir, fname="bars_heavy")
plot_bars_by_segment(ab_new_pd,   "is_new_customer", metrics_cols, title="Médias por segmento", outdir=figdir, fname="bars_new")
plot_bars_by_segment(ab_plat_pd,  "origin_platform", metrics_cols, title="Médias por segmento", outdir=figdir, fname="bars_platform")

# 2) Boxplots por segmento (com e sem clipping p/ reduzir outliers visuais)
for metric in ["monetary", "frequency", "aov_user"]:
    plot_box_by_segment(users_pdf, "heavy_user", metric, clip_p=None, title=f"Boxplot {metric} por heavy_user", outdir=figdir, fname=f"box_heavy")
    plot_box_by_segment(users_pdf, "heavy_user", metric, clip_p=0.01, title=f"Boxplot {metric} por heavy_user (winsor 1%)", outdir=figdir, fname=f"box_heavy_w")

# 3) Histogramas por segmento (distribuições lado a lado, com clipping opcional)
for metric in ["monetary", "frequency", "aov_user"]:
    plot_hist_by_segment(users_pdf, "is_new_customer", metric, bins=40, clip_p=0.01, title=f"Hist {metric} por is_new_customer", outdir=figdir, fname=f"hist_new")
    plot_hist_by_segment(users_pdf, "origin_platform", metric, bins=40, clip_p=0.01, title=f"Hist {metric} por origin_platform", outdir=figdir, fname=f"hist_platform")


### Testes e métricas robustas por segmento

In [ ]:
from src.analysis_ab import collect_user_level_for_tests
from src.analysis_segments import robust_metrics_by_segment, nonparam_tests_by_segment, finance_by_segment

SEG_COLS = [c for c in ["heavy_user","is_new_customer","origin_platform","rfm_segment"] if c in users_pdf.columns]

robust_tables = {}
mw_tests = {}
finance_tables = {}

for seg_col in SEG_COLS:
    robust_tables[seg_col]  = robust_metrics_by_segment(users_pdf, segment_col=seg_col, heavy_threshold=3)
    mw_tests[seg_col]       = nonparam_tests_by_segment(users_pdf, segment_col=seg_col)
    finance_tables[seg_col] = finance_by_segment(users_pdf, segment_col=seg_col, take_rate=0.23, coupon_cost=10.0, redemption_rate=0.30)

# Imprimir exemplo
display(robust_tables["heavy_user"])
mw_tests["heavy_user"], list(finance_tables["heavy_user"].items())[:2]

for seg_col, df in robust_tables.items():
    save_table_csv(df, outdir, f"robust_{seg_col}")
